In [1]:
# Import relevant packages
import pandas as pd
import numpy as np
import pyreadr
from sklearn import preprocessing
import patsy

from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

import hdmpy
import numpy as np
import random
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors

In [2]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import itertools
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_categorical_dtype
from itertools import compress
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.feature_selection import SelectFromModel

# Debiased Machine Learning (Python script)

We provide an additional empirical example of partialling-out with Lasso to estimate the regression coefficient $\beta_1$ in the high-dimensional linear regression model:

   Y = $\beta_1$ D +  $\beta_2$'W + $\epsilon$

Specifically, we are interested in how the rates at which economies of different countries grow ($Y$) are related to the initial wealth levels in each country ($D$) controlling for country's institutional, educational, and other similar characteristics ($W$).

The relationship is captured by $\beta_1$, the speed of convergence/divergence, which measures the speed at which poor countries catch up $(\beta_1< 0)$ or fall behind $(\beta_1> 0)$ rich countries, after controlling for $W$. Our inference question here is: do poor countries grow faster than rich countries, controlling for educational and other characteristics? In other words, is the speed of convergence negative: $ \beta_1 <0?$ This is the Convergence Hypothesis predicted by the Solow Growth Model. This is a structural economic model. Under some strong assumptions, that we won't state here, the predictive exercise we are doing here can be given causal interpretation.

The outcome $Y$ is the realized annual growth rate of a country's wealth (Gross Domestic Product per capita). The target regressor ($D$) is the initial level of the country's wealth. The target parameter $\beta_1$ is the speed of convergence, which measures the speed at which poor countries catch up with rich countries. The controls ($W$) include measures of education levels, quality of institutions, trade openness, and political stability in the country.

## Data analysis

We use the data set GrowthData which is included in the package hdm.

In [3]:
# We downloaded the data that the author used
growth_read = pyreadr.read_r("d:/Users/Manuela/Documents/GitHub/ECO224/Labs/data/GrowthData.RData")

# Extracting the data frame from rdata_read
growth = growth_read[ 'GrowthData' ]
list(growth)

['Outcome',
 'intercept',
 'gdpsh465',
 'bmp1l',
 'freeop',
 'freetar',
 'h65',
 'hm65',
 'hf65',
 'p65',
 'pm65',
 'pf65',
 's65',
 'sm65',
 'sf65',
 'fert65',
 'mort65',
 'lifee065',
 'gpop1',
 'fert1',
 'mort1',
 'invsh41',
 'geetot1',
 'geerec1',
 'gde1',
 'govwb1',
 'govsh41',
 'gvxdxe41',
 'high65',
 'highm65',
 'highf65',
 'highc65',
 'highcm65',
 'highcf65',
 'human65',
 'humanm65',
 'humanf65',
 'hyr65',
 'hyrm65',
 'hyrf65',
 'no65',
 'nom65',
 'nof65',
 'pinstab1',
 'pop65',
 'worker65',
 'pop1565',
 'pop6565',
 'sec65',
 'secm65',
 'secf65',
 'secc65',
 'seccm65',
 'seccf65',
 'syr65',
 'syrm65',
 'syrf65',
 'teapri65',
 'teasec65',
 'ex1',
 'im1',
 'xr65',
 'tot1']

In [4]:
growth.shape[1]

63

### Preprocessing
To account for heterogeneity across counties and time trends in all variables, we remove from them county-specific and time-specific effects in the following preprocessing.

In [5]:
import re

In [6]:
#################################  Find Variable Names from Dataset ########################

def varlist( df = None, type_dataframe = [ 'numeric' , 'categorical' , 'string' ],  pattern = "", exclude = None ):
    varrs = []
    
    if ('numeric' in type_dataframe):
        varrs = varrs + df.columns[ df.apply( is_numeric_dtype , axis = 0 ).to_list() ].tolist()
    
    if ('categorical' in type_dataframe):
        varrs = varrs + df.columns[ df.apply( is_categorical_dtype , axis = 0 ).to_list() ].tolist()
    
    if ('string' in type_dataframe):
        varrs = varrs + df.columns[ df.apply( is_string_dtype , axis = 0 ).to_list() ].tolist()
    
    grepl_result = np.array( [ re.search( pattern , variable ) is not None for variable in df.columns.to_list() ] )
    
    if exclude is None:
        result = list(compress( varrs, grepl_result ) )
    
    else:
        varrs_excluded = np.array( [var in exclude for var in varrs ] )
        and_filter = np.logical_and( ~varrs_excluded ,  grepl_result )
        result = list(compress( varrs, and_filter ) )
    
    return result   

################################# Create Variables ###############################


# Dummy Variables for Year and County Fixed Effects
r = re.compile("X_Jfips")
fixed = list( filter( r.match, growth.columns.to_list() ) )
year = varlist(growth, pattern="X_Tyear")

census = []
census_var = ["^AGE", "^BN", "^BP", "^BZ", "^ED", "^EL","^HI", "^HS", "^INC", "^LF", "^LN", "^PI", "^PO", "^PP", "^PV", "^SPR", "^VS"]

for variable in census_var:
    r = re.compile( variable )
    census = census + list( filter( r.match, growth.columns.to_list() ) )

    
################################ Variables ##################################
# Treatment Variable
d = "gdpsh465"

# Outcome Variable
y = "Outcome"

# Other Control Variables
X1 = ["bmp1l", "freeop", "freetar", "h65", "hm65", "hf65", "p65", "pm65", "pf65", "s65", "sm65", "sf65", "fert65", "mort65", "lifee065", "gpop1", "fert1", "mort1", "invsh41", "geetot1", "geerec1", "gde1"]
X2 = ["govwb1", "govsh41", "gvxdxe41", "high65", "highm65", "highf65", "highc65", "highcm65", "highcf65", "human65", "humanm65", "humanf65", "hyr65", "hyrm65", "hyrf65", "no65", "nom65", "nof65", "pinstab1", "pop65", "worker65", "pop1565", "pop6565", "sec65", "secm65", "secf65", "secc65", "seccm65", "seccf65", "syr65", "syrm65", "syrf65", "teapri65", "teasec65", "ex1", "im1", "xr65", "tot1"]

#################################  Partial out Fixed Effects ########################

# Variables to be Partialled-out
varlist2 = [y] + [d] + X1 + X2 + census

In [7]:
n = growth.shape[0]

We construct the treatment variable, the outcome variable and the matrix $Z$ that includes the control variables

In [8]:
# Treatment Variable
D = growth[ f'{d}']

# Outcome Variable
Y = growth[ f'{y}']

# Construct matrix Z
Z = growth[ X1 + X2 + census ]

Z.shape

(90, 60)

We have in total 60 control variables.

In [9]:
data = pd.concat([Y, D, Z], axis=1)

## OLS without including the country characteristics

In [10]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

In [11]:
# OLS clustering at the County level
model = "Outcome ~ gdpsh465"
baseline_ols = smf.ols(model , data=data).fit()
baseline_ols_table = baseline_ols.summary2().tables[1]
print( baseline_ols_table.iloc[ 1 , 4:] )
baseline_ols_table.iloc[1, :]

[0.025   -0.010810
0.975]    0.013444
Name: gdpsh465, dtype: float64


Coef.       0.001317
Std.Err.    0.006102
t           0.215777
P>|t|       0.829661
[0.025     -0.010810
0.975]      0.013444
Name: gdpsh465, dtype: float64

The point estimate is $0.001317$ with the confidence interval ranging from -0.0108 to 0.0134. This suggests that our estimation is not statistically significant.

## OLS including the country characteristics

To estimate the effect of gun ownership after controlling for a rich set country characteristics we include the controls. First, we estimate the model by OLS and then by an array of the modern regression methods using the double machine learning approach.

In [12]:
# define the variables
y = 'Outcome'

data_columns = list(data)
no_relev_col = ['gdpsh465', 'Outcome']

# This gives us: new_list = ['carrot' , 'lemon']
z = [col for col in data_columns if col not in no_relev_col]

In [13]:
control_formula = "Outcome" + ' ~ ' + 'gdpsh465 + ' + ' + '.join( Z.columns.to_list() )

In [14]:
control_ols = smf.ols( control_formula , data=data).fit()
control_ols_table = control_ols.summary2().tables[1]
print( control_ols_table.iloc[ 1 , 4:] )
control_ols_table.iloc[1, :]

[0.025   -0.070600
0.975]    0.051844
Name: gdpsh465, dtype: float64


Coef.      -0.009378
Std.Err.    0.029888
t          -0.313774
P>|t|       0.756019
[0.025     -0.070600
0.975]      0.051844
Name: gdpsh465, dtype: float64

In this case, the point estimate is  -0.0094  with the confidence interval ranging from -0.0706 to 0.0518. Also suggests that our estimation is not statistically significant.

## DML algorithm

We perform inference of the predictive coefficient $\beta$ in our partially linear statistical model,

using the double machine learning approach.

For $\tilde Y = Y- E(Y|Z)$ and $\tilde D= D- E(D|Z)$, we can write

Using cross-fitting, we employ modern regression methods to build estimators $\hat \ell(Z)$ and $\hat m(Z)$ of $\ell(Z):=E(Y|Z)$ and $m(Z):=E(D|Z)$ to obtain the estimates of the residualized quantities:

Finally, using ordinary least squares of $\tilde Y_i$ on $\tilde D_i$, we obtain the estimate of $\beta$.

The following algorithm comsumes $Y, D, Z$, and a machine learning method for learning the residuals $\tilde Y$ and $\tilde D$, where the residuals are obtained by cross-validation (cross-fitting). Then, it prints the estimated coefficient $\beta$ and the corresponding standard error from the final OLS regression.

In [15]:
def DML2_for_PLM(z, d, y, dreg, yreg, nfold):
    
    # Num ob observations
    nobs = z.shape[0]
    
    # Define folds indices 
    list_1 = [*range(0, nfold, 1)]*nobs
    sample = np.random.choice(nobs,nobs, replace=False).tolist()
    foldid = [list_1[index] for index in sample]

    # Create split function(similar to R)
    def split(x, f):
        count = max(f) + 1
        return tuple( list(itertools.compress(x, (el == i for el in f))) for i in range(count) ) 

    # Split observation indices into folds 
    list_2 = [*range(0, nobs, 1)]
    I = split(list_2, foldid)
    
    # Create array to save errors 
    dtil = np.zeros( len(z) ).reshape( len(z) , 1 )
    ytil = np.zeros( len(z) ).reshape( len(z) , 1 )
    
    # loop to save results
    for b in range(0,len(I)):
    
        # Split data - index to keep are in mask as booleans
        include_idx = set(I[b])  #Here should go I[b] Set is more efficient, but doesn't reorder your elements if that is desireable
        mask = np.array([(i in include_idx) for i in range(len(z))])

        # Lasso regression, excluding folds selected 
        dfit = dreg(z[~mask,], d[~mask,])
        yfit = yreg(z[~mask,], y[~mask,])

        # predict estimates using the 
        dhat = dfit.predict( z[mask,] )
        yhat = yfit.predict( z[mask,] )

        # save errors  
        dtil[mask] =  d[mask,] - dhat.reshape( len(I[b]) , 1 )
        ytil[mask] = y[mask,] - yhat.reshape( len(I[b]) , 1 )
        print(b, " ")
    
    # Create dataframe 
    data_2 = pd.DataFrame(np.concatenate( ( ytil, dtil), axis = 1), columns = ['ytil','dtil'])
   
    # OLS clustering at the County level
    model = "ytil ~ dtil"
    baseline_ols = smf.ols(model , data = data_2 ).fit()
    coef_est = baseline_ols.summary2().tables[1]['Coef.']['dtil']
    se = baseline_ols.summary2().tables[1]['Std.Err.']['dtil']
    
    Final_result = { 'coef_est' : coef_est , 'se' : se , 'dtil' : dtil , 'ytil' : ytil }

    print("Coefficient is {}, SE is equal to {}".format(coef_est, se))
    
    return Final_result

We construct the input matrices

In [16]:
# Create main variables
Y = data['Outcome']
D = data['gdpsh465']
Z = data.drop(['Outcome', 'gdpsh465'], axis=1)

In [17]:
# as matrix
y = Y.to_numpy().reshape( len(Y) , 1 )
d = D.to_numpy().reshape( len(Y) , 1 )
z = Z.to_numpy()

### DML using Lasso to predict y and d

In [18]:
def dreg(z,d):
    alpha=0.00000001
    result = linear_model.Lasso(alpha = alpha).fit(z, d)
    return result

def yreg(z,y):
    alpha=0.00000001
    result = linear_model.Lasso(alpha = alpha).fit(z, y)
    return result

DML2_lasso = DML2_for_PLM(z, d, y, dreg, yreg, 10)

C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.6789721449268009, tolerance: 0.006382639451073899
  model = cd_fast.enet_coordinate_descent(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.01336099615820884, tolerance: 1.8796313891937078e-05
  model = cd_fast.enet_coordinate_descent(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.773868343792369, tolerance: 0.006354746178124679
  model = cd_fast.enet_coordinate_descent(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You mig

0  
1  
2  
3  
4  
5  
6  
7  
8  
9  
Coefficient is 0.009047738016966823, SE is equal to 0.011572274736835954


C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.014293016801366819, tolerance: 2.2767341921353902e-05
  model = cd_fast.enet_coordinate_descent(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.761398192319294, tolerance: 0.006569812192086808
  model = cd_fast.enet_coordinate_descent(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.011882123799571204, tolerance: 1.839301075133571e-05
  model = cd_fast.enet_coordinate_descent(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You

### DML using Post-Lasso to predict y and d

In [19]:
class Lasso_post:
    
    def __init__(self, alpha ):
        self.alpha = alpha

        
    def fit( self, X, Y ):
        self.X = X
        self.Y = Y
        lasso = linear_model.Lasso( alpha = self.alpha ).fit( X , Y )
        model = SelectFromModel( lasso , prefit = True )
        X_new = model.transform( X )
        # Gettin indices from columns which has variance for regression
        index_X = model.get_support()
        
        self.index = index_X
        new_x = X[ : ,  index_X ]
        
        lasso2 = linear_model.Lasso( alpha = self.alpha ).fit( new_x , Y )
        self.model = lasso2
        
        return self
    
    def predict( self , X ):
        
        dropped_X = X[ : , self.index ]
        
        predictions = self.model.predict( dropped_X )
        
        return predictions

This is to verify whether Lasso_post functions as it is expected

In [20]:
def dreg(z,d):
    alpha=0.00000001
    result = Lasso_post( alpha = alpha ).fit( z , d )
    return result

def yreg( z , y ):
    alpha = 0.00000001
    result = Lasso_post( alpha = alpha ).fit( z , y )
    return result

DML2_lasso_post = DML2_for_PLM(z, d, y, dreg, yreg, 10)

C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.0110894801593204, tolerance: 0.006511356023536806
  model = cd_fast.enet_coordinate_descent(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.0158517813810173, tolerance: 0.006511356023536806
  model = cd_fast.enet_coordinate_descent(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.013553993870671995, tolerance: 1.9660016507761837e-05
  model = cd_fast.enet_coordinate_descent(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You m

0  
1  
2  


C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.01542878858946076, tolerance: 2.1103220660100497e-05
  model = cd_fast.enet_coordinate_descent(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7796673796877898, tolerance: 0.006385690735630266
  model = cd_fast.enet_coordinate_descent(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7817562998227644, tolerance: 0.006385690735630266
  model = cd_fast.enet_coordinate_descent(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You mi

3  
4  
5  


C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.012687528117511351, tolerance: 1.9858264991520448e-05
  model = cd_fast.enet_coordinate_descent(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.01268871280013506, tolerance: 1.9858264991520448e-05
  model = cd_fast.enet_coordinate_descent(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.6720666459108369, tolerance: 0.006145698734718423
  model = cd_fast.enet_coordinate_descent(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. Yo

6  
7  
8  
9  
Coefficient is 0.02251863265512348, SE is equal to 0.011062705250905943


C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.815268871726038, tolerance: 0.006566117221351357
  model = cd_fast.enet_coordinate_descent(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.01570313904720193, tolerance: 2.2425726427773657e-05
  model = cd_fast.enet_coordinate_descent(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.016111680692617484, tolerance: 2.2425726427773657e-05
  model = cd_fast.enet_coordinate_descent(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You

#### Lasso y Post- Lasso using hdmy

In [21]:
import hdmpy
from statsmodels.tools import add_constant

In [22]:
class rlasso_hdmy:
    
    def __init__(self, post ):
        self.post = post
       
    def fit( self, X, Y ):
        
        self.X = X
        self.Y = Y
        
        # Standarization of X and Y
        self.rlasso_model = hdmpy.rlasso( X , Y , post = self.post )                
        return self
    
    def predict( self , X_1 ):
        self.X_1 = X_1
        beta = self.rlasso_model.est['coefficients'].to_numpy()
        
        if beta.sum() == 0:
            prediction = np.repeat( self.rlasso_model.est['intercept'] , self.X_1.shape[0] )
        
        else:
            prediction = ( add_constant( self.X_1 , has_constant = 'add') @ beta ).flatten()
                
        return prediction

In [23]:
# Post = false
def dreg(x, d):
    result = rlasso_hdmy( post = False ).fit( x , d )
    return result

def yreg(x,y):
    result = rlasso_hdmy( post = False ).fit( x , y )
    return result

DML2_lasso_hdmpy = DML2_for_PLM(z, d, y, dreg, yreg, 10)

0  
1  
2  
3  
4  
5  
6  
7  
8  
9  
Coefficient is -0.0409409862850038, SE is equal to 0.014664530730975008


In [24]:
# Post = True
def dreg(x, d):
    result = rlasso_hdmy( post = True ).fit( x , d )
    return result

def yreg(x,y):
    result = rlasso_hdmy( post = True ).fit( x , y )
    return result

DML2_lasso_post_hdmpy = DML2_for_PLM(z, d, y, dreg, yreg, 10)

0  
1  
2  
3  
4  
5  
6  
7  
8  
9  
Coefficient is -0.03284115275062999, SE is equal to 0.013664470436882526


In this forms, using hdmy, the coefficients are similar.

#### The model standard_skl_model

In [25]:
class standard_skl_model:
    
    def __init__(self, model ):
        self.model = model
       
    def fit( self, X, Y ):
        
        # Standarization of X and Y
        self.scaler_X = StandardScaler()
        self.scaler_X.fit( X )
        std_X = self.scaler_X.transform( X )
                
        self.model.fit( std_X , Y )
                
        return self
    
    def predict( self , X ):
        
        self.scaler_X = StandardScaler()
        self.scaler_X.fit( X )
        std_X = self.scaler_X.transform( X )
        
        prediction = self.model.predict( std_X )
        
        return prediction

In [26]:
#DML with cross-validated Lasso:
def dreg(z,d):
    result = standard_skl_model( LassoCV(cv = 10 , random_state = 0 ) ).fit( z, d )
    return result

def yreg(z,y):
    result = standard_skl_model( LassoCV(cv = 10 , random_state = 0  ) ).fit( z, y )
    return result

DML2_lasso_cv = DML2_for_PLM(z, d, y, dreg, yreg, 10)

C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.005997325450002311, tolerance: 0.005819745139512947
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0060982709731960405, tolerance: 0.005819745139512947
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the numbe

C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.005751003657164411, tolerance: 0.005464924699741969
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.005587682556349449, tolerance: 0.005464924699741969
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.011921708149983168, tolerance: 0.005464924699741969
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed w

C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.001852239677050527, tolerance: 2.063322266625546e-05
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0018127772316757912, tolerance: 2.063322266625546e-05
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0017539316584990142, tolerance: 2.063322266625546e-05
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective d

C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.411924481770914e-05, tolerance: 1.738843130157323e-05
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0001182738950707557, tolerance: 1.738843130157323e-05
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.67734991202906e-05, tolerance: 1.738843130157323e-05
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective d

C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0012428917372919804, tolerance: 1.738843130157323e-05
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0013459934595976275, tolerance: 1.738843130157323e-05
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0012225357440153288, tolerance: 1.738843130157323e-05
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective 

...

C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.218354667991862e-05, tolerance: 1.765481789979455e-05
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.572669040240398e-05, tolerance: 1.765481789979455e-05
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.405422312928595e-05, tolerance: 1.765481789979455e-05
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective 

C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.00012954435604254494, tolerance: 1.8543337023228266e-05
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0002953685786486922, tolerance: 1.8543337023228266e-05
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0003760900300541735, tolerance: 1.8543337023228266e-05
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Object

C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.00036440189024943004, tolerance: 1.7452508005820558e-05
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.000329394366264002, tolerance: 1.7452508005820558e-05
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.000459756543827787, tolerance: 1.7452508005820558e-05
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objectiv

9  
Coefficient is -0.02159631162268906, SE is equal to 0.01209559087718817


C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.534351872487492e-05, tolerance: 2.1265027193047558e-05
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.444666981581665e-05, tolerance: 2.1265027193047558e-05
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.42046899324389e-05, tolerance: 2.1265027193047558e-05
  model = cd_fast.enet_coordinate_descent_gram(
C:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objectiv

### DML using Elastic Net to predict y and d

In [27]:
# DML with cross-validated Elastic Net:
def dreg(z,d):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.5, max_iter = 100000 ) ).fit( z, d )
    return result

def yreg(z,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.5, max_iter = 100000 ) ).fit( z, y )
    return result

DML2_elnet = DML2_for_PLM(z, d, y, dreg, yreg,10)

C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0  


C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


1  


C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


...

7  


C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


8  


C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


9  
Coefficient is -0.009194044665148064, SE is equal to 0.01245419560909518


### DML using Ridge to predict y and d

In [28]:
#DML with cross-validated Ridge:
def dreg(z,d):
    result = standard_skl_model( ElasticNetCV( cv = 10 ,  random_state = 0 , l1_ratio = 0.0001 ) ).fit( z, d )
    return result

def yreg(z,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.0001 ) ).fit( z, y )
    return result

DML2_ridge = DML2_for_PLM(z, d, y, dreg, yreg, 10)

C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0  


C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


1  


C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


...

7  


C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


8  


C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


9  
Coefficient is -0.01178038966659506, SE is equal to 0.008838942960452081


In [29]:
#DML with OLS:
def dreg(z,d):
    result = LinearRegression().fit( z, d )
    return result

def yreg(z,y):
    result = LinearRegression().fit( z, y )
    return result

DML2_ols = DML2_for_PLM(z, d, y, dreg, yreg, 10)

0  
1  
2  
3  
4  
5  
6  
7  
8  
9  
Coefficient is 0.022607439613684797, SE is equal to 0.016365977119987937


### DML using Random Forest to predict y and d

In [30]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [31]:
#DML with Random Forest:
def dreg(z,d):
    result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, d )
    return result

def yreg(z,y):
    result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, y )
    return result

DML2_RF = DML2_for_PLM(z, d, y, dreg, yreg, 10)   # set to 2 due to computation time

<ipython-input-31-f86bac02d5ff>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, d )
<ipython-input-31-f86bac02d5ff>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, y )


0  


<ipython-input-31-f86bac02d5ff>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, d )
<ipython-input-31-f86bac02d5ff>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, y )


1  


<ipython-input-31-f86bac02d5ff>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, d )
<ipython-input-31-f86bac02d5ff>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, y )


...

7  


<ipython-input-31-f86bac02d5ff>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, d )
<ipython-input-31-f86bac02d5ff>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, y )


8  


<ipython-input-31-f86bac02d5ff>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, d )
<ipython-input-31-f86bac02d5ff>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, y )


9  
Coefficient is -0.042084220382900425, SE is equal to 0.012382720556015322


### The best combination of methods to predict y and d

We compute RMSE for predicting D and Y, and see which of the methods works better

In [32]:
mods = [DML2_ols, DML2_lasso, DML2_lasso_post , DML2_lasso_cv, DML2_ridge, DML2_elnet, DML2_RF]
mods_name = ["DML2_ols", "DML2_lasso", "DML2_lasso_post" , "DML2_lasso_cv", 'DML2_ridge', 'DML2_elnet', 'DML2_RF']

def mdl( model , model_name ):
    
    RMSEY = np.sqrt( np.mean( model[ 'ytil' ] ) ** 2 ) # I have some doubts about these equations...we have to recheck
    RMSED = np.sqrt( np.mean( model[ 'dtil' ] ) ** 2 ) # I have some doubts about these equations...we have to recheck
    
    result = pd.DataFrame( { model_name : [ RMSEY , RMSED ]} , index = [ 'RMSEY' , 'RMSED' ])
    return result

RES = [ mdl( model , name ) for model, name in zip( mods , mods_name ) ]
    

pr_Res = pd.concat( RES, axis = 1)

pr_Res

,DML2_ols,DML2_lasso,DML2_lasso_post,DML2_lasso_cv,DML2_ridge,DML2_elnet,DML2_RF
RMSEY,0.000710,0.001545,0.000838,7.393777e-17,1.387779e-18,1.187322e-17,0.000701
RMSED,0.058557,0.003623,0.028557,1.697408e-15,5.033011e-16,2.072416e-16,0.032100


To verfie that the function DML2_for_PLM has no errors:

In [33]:
np.where(DML2_lasso_post[ 'ytil' ] == 0)[0].size

0

It looks like the best method for predicting D and Y is Ridge:

In [34]:
#DML with the BEST (cross-validated Ridge):
def dreg(z,d):
    result = standard_skl_model( ElasticNetCV( cv = 10 ,  random_state = 0 , l1_ratio = 0.0001 ) ).fit( z, d )
    return result

def yreg(z,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.0001 ) ).fit( z, y )
    return result

DML2_best = DML2_for_PLM(z, d, y, dreg, yreg, 10)

C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0  


C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


1  


C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


...

7  


C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


8  


C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


9  
Coefficient is -0.026484141479602326, SE is equal to 0.009570579426470398


### Results in a table

In [35]:
table = np.zeros( ( 9 , 2 ))
table[ 0 , 0] = baseline_ols_table.iloc[ 1 , 0 ]
table[ 1 , 0] = control_ols_table.iloc[ 1 , 0 ]
table[ 2 , 0] = DML2_lasso['coef_est']
table[ 3 , 0] = DML2_lasso_post['coef_est']
table[ 4 , 0] = DML2_lasso_cv['coef_est']
table[ 5 , 0] = DML2_ridge['coef_est']
table[ 6 , 0] = DML2_elnet['coef_est']
table[ 7 , 0] = DML2_RF['coef_est']
table[ 8 , 0] = DML2_best['coef_est']
table[ 0 , 1] = baseline_ols_table.iloc[ 1 , 1 ]
table[ 1 , 1] = control_ols_table.iloc[ 1 , 1 ]
table[ 2 , 1] = DML2_lasso['se']
table[ 3 , 1] = DML2_lasso_post['se']
table[ 4 , 1] = DML2_lasso_cv['se']
table[ 5 , 1] = DML2_ridge['se']
table[ 6 , 1] = DML2_elnet['se']
table[ 7 , 1] = DML2_RF['se']
table[ 8 , 1] = DML2_best['se']

In [36]:
table = pd.DataFrame( table , index = [ "Baseline OLS", "Least Squares with controls", "Lasso", \
             "Post-Lasso", "CV Lasso","CV Elnet", "CV Ridge", "Random Forest", "Best" ] , \
            columns = ["Estimate","Standard Error"] )
table.round( 3 )

,Estimate,Standard Error
Baseline OLS,0.001,0.006
Least Squares with controls,-0.009,0.030
Lasso,0.009,0.012
Post-Lasso,0.023,0.011
CV Lasso,-0.022,0.012
CV Elnet,-0.012,0.009
CV Ridge,-0.009,0.012
Random Forest,-0.042,0.012
Best,-0.026,0.010


In [37]:
table.to_latex()

'\\begin{tabular}{lrr}\n\\toprule\n{} &  Estimate &  Standard Error \\\\\n\\midrule\nBaseline OLS                &  0.001317 &        0.006102 \\\\\nLeast Squares with controls & -0.009378 &        0.029888 \\\\\nLasso                       &  0.009048 &        0.011572 \\\\\nPost-Lasso                  &  0.022519 &        0.011063 \\\\\nCV Lasso                    & -0.021596 &        0.012096 \\\\\nCV Elnet                    & -0.011780 &        0.008839 \\\\\nCV Ridge                    & -0.009194 &        0.012454 \\\\\nRandom Forest               & -0.042084 &        0.012383 \\\\\nBest                        & -0.026484 &        0.009571 \\\\\n\\bottomrule\n\\end{tabular}\n'